In [1]:
import graphlab

In [2]:
song_data = graphlab.SFrame('song_data.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to jjmmg@yahoo.comand will expire on October 22, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5984 - Server binary: C:\Users\JJMMG\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\JJMMG\AppData\Local\Temp\graphlab_server_1446137541.log.0
[INFO] GraphLab Server Version: 1.6.1


In [6]:
print " Question 1: Kanye West(%s), Foo Fighters(%s), Taylor Swift(%s), Lady GaGa(%s)" % (len(song_data[song_data['artist'] == 'Kanye West'].unique()), len(song_data[song_data['artist'] == 'Foo Fighters'].unique()), len(song_data[song_data['artist'] == 'Taylor Swift'].unique()), len(song_data[song_data['artist'] == 'Lady GaGa'].unique()))

 Question 1: Kanye West(3775), Foo Fighters(3429), Taylor Swift(6227), Lady GaGa(4129)


In [34]:
artists_songs = song_data[(song_data['artist'] == 'Kanye West') | (song_data['artist'] == 'Foo Fighters') | (song_data['artist'] == 'Taylor Swift') | (song_data['artist'] == 'Lady GaGa')].unique()
artists_songs = artists_songs.groupby(key_columns='artist', operations={'count': graphlab.aggregate.COUNT()})
print 'Question 1: %s' % (artists_songs.sort('count',ascending=False)[0]['artist'])

Question 1: Taylor Swift


In [37]:
artists_songs = song_data.groupby(key_columns='artist', operations={'total_count': graphlab.aggregate.SUM('listen_count')})
print 'Question 2: %s' % (artists_songs.sort('total_count',ascending=False)[0]['artist'])
print 'Question 3: %s' % (artists_songs.sort('total_count',ascending=False)[-1]['artist'])

Question 2: Kings Of Leon
Question 3: William Tabbert


In [38]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [39]:
personalized_model = graphlab.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Ignoring columns song_id, listen_count, title, artist;
PROGRESS:     To use one of these as a target column, set target = <column_name>
PROGRESS:     and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 893580 observations with 66085 users and 9952 items.
PROGRESS:     Data prepared in: 1.56507s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 9952 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 2.69503         |
PROGRESS: | 2000            | 2.80404         |
PROGRESS: | 3000            | 2.90504         |
PROGRESS: | 4000            | 3.00205         |
PROGRESS: | 5000            | 3.10406         |
PROGRESS: | 6000            | 3.21406         |
PROGRESS: | 7000            | 3.30607         |


In [40]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [43]:
recommendations = personalized_model.recommend(subset_test_users,k=1)

PROGRESS: recommendations finished on 1000/10000 queries. users per second: 1418.39
PROGRESS: recommendations finished on 2000/10000 queries. users per second: 1387.87
PROGRESS: recommendations finished on 3000/10000 queries. users per second: 1398.54
PROGRESS: recommendations finished on 4000/10000 queries. users per second: 1386.42
PROGRESS: recommendations finished on 5000/10000 queries. users per second: 1380
PROGRESS: recommendations finished on 6000/10000 queries. users per second: 1388.82
PROGRESS: recommendations finished on 7000/10000 queries. users per second: 1397.14
PROGRESS: recommendations finished on 8000/10000 queries. users per second: 1400
PROGRESS: recommendations finished on 9000/10000 queries. users per second: 1407.94
PROGRESS: recommendations finished on 10000/10000 queries. users per second: 1396.58


In [46]:
print 'Question 4: %s' % (recommendations.groupby(key_columns='song', operations={'count': graphlab.aggregate.COUNT()}).sort('total_count',ascending=False)[0]['artist'])



song,count
The Climb - Miley Cyrus,3
Hey Daddy (Daddy's Home)- Usher ...,6
I Gotta Feeling - BlackEyed Peas ...,13
Cerdo - Molotov,1
Ich Will - Rammstein,9
Too Deep - Girl Talk,3
Dumpweed - Blink-182,2
Guys Like Me - EricChurch ...,2
I Can't Love You Back -Easton Corbin ...,2
Freedom - Akon,4


In [ ]:
personalized_model.recommend(users=[users[0]])

In [ ]:
personalized_model.recommend(users=[users[1]])

###We can also apply the model to find similar songs to any song in the dataset

In [ ]:
personalized_model.get_similar_items(['With Or Without You - U2'])

In [ ]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

#Quantitative comparison between the models

We now formally compare the popularity and the personalized models using precision-recall curves. 

In [ ]:
if graphlab.version[:3] >= "1.6":
    model_performance = graphlab.compare(test_data, [popularity_model, personalized_model], user_sample=0.05)
    graphlab.show_comparison(model_performance,[popularity_model, personalized_model])
else:
    %matplotlib inline
    model_performance = graphlab.recommender.util.compare_models(test_data, [popularity_model, personalized_model], user_sample=.05)

The curve shows that the personalized model provides much better performance. 